## Initialization

In [ ]:
!apt install python3.10-venv
!git clone https://git.ecker.tech/mrq/ai-voice-cloning/
%cd /content/ai-voice-cloning
# get local dependencies
!git submodule init
!git submodule update --remote
# setup venv
!python3 -m venv venv
!source ./venv/bin/activate
!python3 -m pip install --upgrade pip # just to be safe
# CUDA
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# install requirements
!python3 -m pip install -r ./modules/tortoise-tts/requirements.txt # install TorToiSe requirements
!python3 -m pip install -e ./modules/tortoise-tts/ # install TorToiSe
!python3 -m pip install -r ./modules/dlas/requirements.txt # instal DLAS requirements, last, because whisperx will break a dependency here
!python3 -m pip install -e ./modules/dlas/ # install DLAS
!python3 -m pip install -r ./requirements.txt # install local requirements

# Update Repos

In [ ]:
# for my debugging purposes
%cd /content/ai-voice-cloning/
!./update.sh

# Mount Drive

In [ ]:
# only run once, this will save all userdata to your Drive
# it shouldn't delete through symlinks, but you never know

from google.colab import drive

%cd /content/ai-voice-cloning
drive.flush_and_unmount()
!rm -r ./{training,results,voices,config}
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/ai-voice-cloning/
!mv /content/drive/MyDrive/{training,results,voices,config} /content/drive/MyDrive/ai-voice-cloning
!mkdir /content/drive/MyDrive/ai-voice-cloning/{training,results,voices,config}
!ln -s /content/drive/MyDrive/ai-voice-cloning/{training,results,voices,config} ./

## Running

In [ ]:
%cd /content/ai-voice-cloning/
!source ./venv/bin/activate
!python3 ./src/main.py --share

# Restart Runtime

In [ ]:
exit()

# Fallback Training

In [ ]:
# This is in case you can't get training through the web UI
%cd /content/ai-voice-cloning
!python ./dlas/codes/train.py -opt ./training/finetune.yaml

## Exporting

In [ ]:
# if you're not using drive mounting
%cd /content/ai-voice-cloning
!apt install -y p7zip-full
from datetime import datetime
timestamp = datetime.now().strftime('%m-%d-%Y_%H:%M:%S')
!mkdir -p "../{timestamp}/results"
!mv ./results/* "../{timestamp}/results/."
!mv ./training/* "../{timestamp}/training/."
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on "../{timestamp}.7z" "../{timestamp}/"
!ls ~/
!echo "Finished zipping, archive is available at {timestamp}.7z"